# Preparing the LLM

In [13]:
import dspy

llm = dspy.OpenAI(
    model_type="chat",
    # model="groq/llama3-70b-8192",
    # model="azure/gpt-35-turbo-1106",
    # model="openai/gpt-3.5-turbo",
    model="gpt-3.5-turbo",
    # model="anthropic/claude-3-haiku-20240307",
    # api_base="http://localhost:8080/proxy/v1/",
    max_tokens=2048,
    temperature=0,
)

print("LLM test response:", llm("hello there"))

colbertv2_wiki17_abstracts = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.settings.configure(lm=llm, rm=colbertv2_wiki17_abstracts)

LLM test response: ['Hello! How can I assist you today?']


# Preparing the Dataset

In [2]:
from dspy.datasets import HotPotQA

# Load the dataset.
dataset = HotPotQA(train_seed=1, train_size=20, eval_seed=2023, dev_size=50, test_size=0)

# Tell DSPy that the 'question' field is the input. Any other fields are labels and/or metadata.
trainset = [x.with_inputs('question') for x in dataset.train]
devset = [x.with_inputs('question') for x in dataset.dev]

len(trainset), len(devset)

/Users/rchaves/Projects/langwatch-saas/langwatch/python-sdk/.venv/lib/python3.9/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


(20, 50)

# Defining the model

In [3]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with short factoid answers."""

    context = dspy.InputField(desc="may contain relevant facts")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="often between 1 and 5 words")


class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()

        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)

    def forward(self, question):
        context = self.retrieve(question).passages
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)


dev_example = devset[18]
print(f"[Devset] Question: {dev_example.question}")
print(f"[Devset] Answer: {dev_example.answer}")
print(f"[Devset] Relevant Wikipedia Titles: {dev_example.gold_titles}")

generate_answer = RAG()

pred = generate_answer(question=dev_example.question)

# Print the input and the prediction.
print(f"[Prediction] Question: {dev_example.question}")
print(f"[Prediction] Predicted Answer: {pred.answer}")

[Devset] Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
[Devset] Answer: English
[Devset] Relevant Wikipedia Titles: {'Restaurant: Impossible', 'Robert Irvine'}
[Prediction] Question: What is the nationality of the chef and restaurateur featured in Restaurant: Impossible?
[Prediction] Predicted Answer: British


# Login to LangWatch

In [4]:
import langwatch

langwatch.endpoint = "http://localhost:3000"
langwatch.login()

LangWatch API key is already set, if you want to login again, please call as langwatch.login(relogin=True)


# Start Training Session!

In [20]:
import sys
import importlib

if "langwatch" in sys.modules:
    del sys.modules["langwatch"]
if "langwatch.dspy" in sys.modules:
    del sys.modules["langwatch.dspy"]

import langwatch
import json
from langwatch.dspy import SerializableAndPydanticEncoder





from dspy.teleprompt import BootstrapFewShotWithRandomSearch, LabeledFewShot, BootstrapFewShot
import dspy.evaluate

# make logger appear on jupyter notebook
dspy.logger.info = print

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic optimizer, which will compile our RAG program.
# optimizer = BootstrapFewShotWithRandomSearch(metric=validate_context_and_answer, max_rounds=1)
optimizer = BootstrapFewShot(metric=validate_context_and_answer, max_bootstrapped_demos=10, max_labeled_demos=10, max_rounds=3)
# optimizer.num_candidate_sets = 0

langwatch.dspy.init(experiment="dspy-visualizer-example", optimizer=optimizer)

# Compile!
compiled_rag = optimizer.compile(RAG(), trainset=trainset)

Experiment initialized, open http://localhost:3000/inbox-narrator/experiments/dspy-visualizer-example to track your DSPy training session live



100%|██████████| 20/20 [00:14<00:00,  1.38it/s]


Bootstrapped 8 full traces after 20 examples in round 2.


In [9]:
optimizer.__dict__

{'metric': <function __main__.validate_context_and_answer(example, pred, trace=None)>,
 'metric_threshold': None,
 'teacher_settings': {},
 'max_bootstrapped_demos': 3,
 'max_labeled_demos': 0,
 'max_rounds': 1,
 'max_errors': 5,
 'error_count': 0,
 'error_lock': <unlocked _thread.lock object at 0x156b01a20>,
 'trainset': [Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'}),
  Example({'question': 'which  American actor was Candace Kita  guest starred with ', 'answer': 'Bill Murray'}) (input_keys={'question'}),
  Example({'question': 'Which of these publications was most recently published, Who Put the Bomp or Self?', 'answer': 'Self'}) (input_keys={'question'}),
  Example({'question': 'The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?', 'answer': '1950'}) (input_keys={'question'}),
  Example({'question': 'Which magazine has published a

In [27]:
compiled_rag(question="what is the capital of brazil??")

Prediction(
    context=['Capitals of Brazil | The current Capital of Brazil (Brasil) is Brasília. The city of Salvador is usually called the "first capital of Brazil" for being more than two centuries the seat for the Portuguese colonial administration in Brazil (general government and viceroyalty).', 'Brasilia (disambiguation) | Brasília is the capital of Brazil.', "Brasília | Brasília (] ) is the federal capital of Brazil and seat of government of the Federal District. The city is located atop the Brazilian highlands in the country's center-western region. It was founded on April 21, 1960, to serve as the new national capital. Brasília and its metro area were estimated to be Brazil's 4th most populous city. Among major Latin American cities, Brasília has the highest GDP per capita at R$ ()."],
    answer='Brasília'
)

In [24]:
llm.inspect_history()




Answer questions with short factoid answers.

---

Question: This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?
Answer: The Waltz King

Question: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?
Answer: 1950

---

Follow the following format.

Context: may contain relevant facts

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: often between 1 and 5 words

---

Context:
[1] «Tae Kwon Do Times | Tae Kwon Do Times is a magazine devoted to the martial art of taekwondo, and is published in the United States of America. While the title suggests that it focuses on taekwondo exclusively, the magazine also covers other Korean martial arts. "Tae Kwon Do Times" has published articles by a wide range of authors, including He-Young Kimm, Thomas Kurz, Scott Shaw, and Mark Van Schuyver.»
[2] «Kwon Tae-man | Kwon

'\n\n\nAnswer questions with short factoid answers.\n\n---\n\nQuestion: This American guitarist best known for her work with the Iron Maidens is an ancestor of a composer who was known as what?\nAnswer: The Waltz King\n\nQuestion: The Victorians - Their Story In Pictures is a documentary series written by an author born in what year?\nAnswer: 1950\n\n---\n\nFollow the following format.\n\nContext: may contain relevant facts\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: often between 1 and 5 words\n\n---\n\nContext:\n[1] «Tae Kwon Do Times | Tae Kwon Do Times is a magazine devoted to the martial art of taekwondo, and is published in the United States of America. While the title suggests that it focuses on taekwondo exclusively, the magazine also covers other Korean martial arts. "Tae Kwon Do Times" has published articles by a wide range of authors, including He-Young Kimm, Thomas Kurz, Scott Shaw, and Mark Van Schuy

In [ ]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch, LabeledFewShot, BootstrapFewShot
import dspy.evaluate

# make logger appear on jupyter notebook
dspy.logger.info = print

def validate_context_and_answer(example, pred, trace=None):
    answer_EM = dspy.evaluate.answer_exact_match(example, pred)
    answer_PM = dspy.evaluate.answer_passage_match(example, pred)
    return answer_EM and answer_PM

# Set up a basic optimizer, which will compile our RAG program.
optimizer = BootstrapFewShotWithRandomSearch(metric=langwatch.dspy.track_metric(validate_context_and_answer), max_rounds=1)
# optimizer = BootstrapFewShot(metric=validate_context_and_answer, max_rounds=1)
# optimizer.num_candidate_sets = 0

# Compile!
compiled_rag = optimizer.compile(RAG(), trainset=trainset)

In [ ]:
len(trainset)

20

In [ ]:
from dspy.evaluate.evaluate import Evaluate

# Set up the `evaluate_on_hotpotqa` function. We'll use this many times below.
evaluate_on_hotpotqa = Evaluate(devset=devset, num_threads=1, display_progress=True, display_table=5)

# Evaluate the `compiled_rag` program with the `answer_exact_match` metric.
metric = dspy.evaluate.answer_exact_match
evaluate_on_hotpotqa(compiled_rag, metric=metric)

Average Metric: 5 / 8  (62.5):  16%|█▌        | 8/50 [00:16<01:35,  2.27s/it]

KeyboardInterrupt: 